In [31]:
import pandas as pd #To enable viewing the info as table and manipulate the info in the tables
from bs4 import BeautifulSoup #To enable use of the HTML of the page
import requests #Enable access content on the webpage

In [32]:
#headers will help in evading detection as a scraper
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
##The URL we want is specified to requests.get, and the headers parsed an as argument
requested =requests.get('https://www.amazon.com/gp/bestsellers/books/ref=bsm_nav_pill_print/ref=s9_acss_bw_cg_bsmpill_1c1_w?pf_rd_m=ATVPDKIKX0DER&pf_rd_s=merchandised-search-1&pf_rd_r=JSFR919BB1373W4FETRV&pf_rd_t=101&pf_rd_p=65e3ce24-654c-43fb-a17b-86a554348820&pf_rd_i=16857165011', headers=headers)#, proxies=proxies)
content = requested.content #Stores the content in the URL into content
soup = BeautifulSoup(content, "html.parser")#This pulls out all the HTML in the webpage
# print(soup)
Book_list = [] #List for the books
#Remember, soup has the HTML content of the page. We want to retrieve our data by calling the specific tags they fall under
for item in soup.findAll('div', attrs={'class':'a-section a-spacing-none aok-relative'}):
    book = item.find('span', attrs={'class':'zg-text-center-align'}) #Finds the name of the book
    book_name = book.find_all('img', alt=True)
    book_author = item.find('a', attrs={'class':'a-size-small a-link-child'})#Finds the book author by retrieving the content of this tag
    book_rating = item.find('span', attrs={'class':'a-icon-alt'})#Finds the book rating  by retrieving the content of this tag
    no_of_users_rated = item.find('a', attrs={'class':'a-size-small a-link-normal'}) #Finds the number of users that reviewed a book
    price = item.find('span', attrs={'class':'p13n-sc-price'})

    list_for_individual_book = []
    if book is not None: #Checks whether there is a book name
        list_for_individual_book.append(book_name[0]['alt'])#appends the book name to the list
    else:
        list_for_individual_book.append("No name book") #What will appear in the list if there is no book name

    if book_author is not None: #checks whether there is a book author
        list_for_individual_book.append(book_author.text)
    elif book_author is None:#If there is no no book author
        author = item.find('span', attrs={'class': 'a-size-small a-color-base'})#Provides an alternate option: the publication
        if author is not None: #If the alternate location is not empty
            list_for_individual_book.append(author.text)#appends authors name to list
        else: #When both of the above have proved futile
            list_for_individual_book.append("No author") #This is what will be appended if there is no author\

    if book_rating is not None:#Checks that the variable is not empty
        list_for_individual_book.append(book_rating.text)
    else: #If it is empty
        list_for_individual_book.append("No rating")

    if no_of_users_rated is not None:#Checks that the variable is not empty
        list_for_individual_book.append(no_of_users_rated.text)#Appends to the list
    else:#If it is empty
        list_for_individual_book.append("0")#Appends zero

    if price is not None:#Checks that the variable is not empty
        list_for_individual_book.append(price.text)#Appends to the list
    else:#If it is empty
        list_for_individual_book.append("0")#Appends zero
    #We now need to append all this info about a book to the main list: Book_list
    Book_list.append(list_for_individual_book)
print(Book_list)    

[['Atlas of the Heart: Mapping Meaningful Connection and the Language of Human Experience', 'Brené Brown', '4.6 out of 5 stars', '72', '$19.48'], ['Big Shot Diary of a Wimpy Kid Book 16', 'Jeff Kinney', '4.8 out of 5 stars', '2,564', '$7.49'], ['The Real Anthony Fauci: Bill Gates, Big Pharma, and the Global War on Democracy and Public Health (Children’s Health…', 'Robert F. Kennedy Jr.', '4.9 out of 5 stars', '1,607', '$20.68'], ['The Screaming Goat (Book & Figure)', 'No author', '4.7 out of 5 stars', '21,797', '$7.75'], ['It Ends with Us: A Novel', 'Colleen Hoover', '4.7 out of 5 stars', '41,507', '$9.47'], ['Johnny the Walrus', 'Matt Walsh', '4.6 out of 5 stars', '97', '$20.00'], ['American Marxism', 'Mark R. Levin', '4.9 out of 5 stars', '21,945', '$14.00'], ['The Storyteller: Tales of Life and Music', 'Dave Grohl', '4.8 out of 5 stars', '3,964', '$17.99'], ['Cat Kid Comic Club: Perspectives: A Graphic Novel (Cat Kid Comic Club #2): From the Creator of Dog Man', 'Dav Pilkey', '4.8 o

In [33]:
#Next, pandas will be used to put the data into a CSV and modify some columns it to enable sorting of the prices
#Loading the list to a dataframe and naming the columns
dataframe = pd.DataFrame(Book_list,columns=["Name_of_Book", "Author",'Book_Rating',"No_of_ratings",'price'])
#Upload it to a CSV file using to_csv function
dataframe.to_csv("Best_Selling_Books_Amazon.csv")
#Now that we have created the file, we need to read it so as to access the data within
df = pd.read_csv("Best_Selling_Books_Amazon.csv")
#######Let's inspect the data to see how it looks. There should be 100 books, thus 100 rows
df.shape
df.head(10)#See how the data looks


,Unnamed: 0,Name_of_Book,Author,Book_Rating,No_of_ratings,price
0,0,Atlas of the Heart: Mapping Meaningful Connect...,Brené Brown,4.6 out of 5 stars,72,$19.48
1,1,Big Shot Diary of a Wimpy Kid Book 16,Jeff Kinney,4.8 out of 5 stars,"2,564",$7.49
2,2,"The Real Anthony Fauci: Bill Gates, Big Pharma...",Robert F. Kennedy Jr.,4.9 out of 5 stars,"1,607",$20.68
3,3,The Screaming Goat (Book & Figure),No author,4.7 out of 5 stars,"21,797",$7.75
4,4,It Ends with Us: A Novel,Colleen Hoover,4.7 out of 5 stars,"41,507",$9.47
5,5,Johnny the Walrus,Matt Walsh,4.6 out of 5 stars,97,$20.00
6,6,American Marxism,Mark R. Levin,4.9 out of 5 stars,"21,945",$14.00
7,7,The Storyteller: Tales of Life and Music,Dave Grohl,4.8 out of 5 stars,"3,964",$17.99
8,8,Cat Kid Comic Club: Perspectives: A Graphic No...,Dav Pilkey,4.8 out of 5 stars,117,$8.62
9,9,Interesting Stories For Curious People: A Coll...,Bill O'Neill,4.5 out of 5 stars,"10,839",$8.19


In [34]:
#Since we are looking for the most expensive books from those with the most reveiws, I will have to make some changes to
#price formatting and additionally make edits to no_of_ratings to remove the commas. 
df["price"] = df["price"].str.replace('$', '')#Change it to string and replace $ with space
df['price'] = df['price'].astype(float)#Convert it to a manipulatable data type
df["No_of_ratings"] = df["No_of_ratings"].str.replace(',', '')
df['No_of_ratings'] = pd.to_numeric(df['No_of_ratings'], errors='ignore')
df.dtypes

<ipython-input-34-be4065c23612>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df["price"] = df["price"].str.replace('$', '')#Change it to string and replace $ with space


Unnamed: 0         int64
Name_of_Book      object
Author            object
Book_Rating       object
No_of_ratings      int64
price            float64
dtype: object

In [35]:
df = df.sort_values(["No_of_ratings"], axis=0, ascending=False)##sorted the price in descending order and only shows the first 10
#Here, I will take the top 30 books with the highest reviews, then sort their prices
sorted_ratings = df.head(30)
sorted_ratings

,Unnamed: 0,Name_of_Book,Author,Book_Rating,No_of_ratings,price
44,44,"The Boy, the Mole, the Fox and the Horse",Charlie Mackesy,4.8 out of 5 stars,86821,10.35
34,34,My First Learn to Write Workbook: Practice for...,Crystal Radke,4.8 out of 5 stars,63886,4.53
13,13,Harry Potter Paperback Box Set (Books 1-7),J. K. Rowling,4.9 out of 5 stars,58131,35.00
22,22,If Animals Kissed Good Night,Ann Whitford Paul,4.8 out of 5 stars,57948,4.44
10,10,Atomic Habits: An Easy & Proven Way to Build G...,James Clear,4.8 out of 5 stars,56535,11.98
21,21,I Love You to the Moon and Back,Amelia Hepworth,4.9 out of 5 stars,44303,1.55
37,37,The Very Hungry Caterpillar,Eric Carle,4.9 out of 5 stars,42548,5.06
4,4,It Ends with Us: A Novel,Colleen Hoover,4.7 out of 5 stars,41507,9.47
31,31,The Seven Husbands of Evelyn Hugo: A Novel,Taylor Jenkins Reid,4.6 out of 5 stars,37828,4.95
48,48,My First Library : Boxset of 10 Board Books fo...,Wonder House Books,4.6 out of 5 stars,33647,16.93


In [36]:
#Here, the prices of the top 30 books with the highest reviews have been sorted by price and the top 10 returned. 
pricing_sort = sorted_ratings.sort_values(["price"], axis=0, ascending=False)
sorted_price = pricing_sort.head(10)#
sorted_price

,Unnamed: 0,Name_of_Book,Author,Book_Rating,No_of_ratings,price
13,13,Harry Potter Paperback Box Set (Books 1-7),J. K. Rowling,4.9 out of 5 stars,58131,35.00
49,49,Percy Jackson and the Olympians 5 Book Paperba...,Rick Riordan,4.9 out of 5 stars,26988,18.80
24,24,The Lincoln Highway: A Novel,Amor Towles,4.5 out of 5 stars,11018,18.28
48,48,My First Library : Boxset of 10 Board Books fo...,Wonder House Books,4.6 out of 5 stars,33647,16.93
45,45,The Wish,Nicholas Sparks,4.8 out of 5 stars,11190,14.00
6,6,American Marxism,Mark R. Levin,4.9 out of 5 stars,21945,14.00
41,41,How to Draw Cool Stuff: A Drawing Guide for Te...,Catherine V Holmes,4.7 out of 5 stars,13608,13.99
20,20,From Crook to Cook: Platinum Recipes from Tha ...,Snoop Dogg,4.8 out of 5 stars,20545,12.74
10,10,Atomic Habits: An Easy & Proven Way to Build G...,James Clear,4.8 out of 5 stars,56535,11.98
44,44,"The Boy, the Mole, the Fox and the Horse",Charlie Mackesy,4.8 out of 5 stars,86821,10.35
